In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"
#os.environ["CUDA_VISIBLE_DEVICES"]=""
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

import datetime

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

%load_ext autoreload
%autoreload 2

from freedom.utils.dataset import Data
from freedom.utils.callback import Save

from freedom.neural_nets.hitnet import get_hitnet

In [2]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [3]:
data = Data(dirs=['/home/iwsatlas1/peller/work/oscNext/level3_v01.03/140000_i3cols','/home/iwsatlas1/peller/work/oscNext/level3_v01.03/120000_i3cols'])

In [18]:
hitnet = get_hitnet(data.labels)
hitnet.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 9)]          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
hitnet_trafo_1 (hitnet_trafo)   (None, 17)           0           input_3[0][0]                    
__________________________________________________________________________________________________
dense_12 (Dense)                (None, 32)           576         hitnet_trafo_1[0][0]             
____________________________________________________________________________________________

In [19]:
optimizer = tf.keras.optimizers.Adam(0.00001)
hitnet.compile(loss='binary_crossentropy', optimizer=optimizer)

In [20]:
train_id = 'HitNet_shuffle_batches' + datetime.datetime.now().strftime("%d_%b_%Y-%Hh%M")

In [7]:
callbacks = []
callbacks.append(Save(save_every=2, path_template='../../resources/models/'+train_id+'/epoch_%i'))
callbacks.append(tf.keras.callbacks.TensorBoard(log_dir='../../resources/logs/'+train_id, histogram_freq=1))

In [8]:
train_data, test_data = data.get_hitnet_data(train_batch_size=4096,
                                             test_batch_size=4096,
                                             )

In [21]:
d_train_data = train_data.apply(tf.data.experimental.prefetch_to_device(':'.join(tf.config.list_physical_devices(device_type="GPU")[0].name.split(':')[1:])))

In [ ]:
hist = hitnet.fit(d_train_data, epochs=1000, verbose=2, callbacks=callbacks, validation_data=test_data,
                  initial_epoch=len(callbacks[0].hist['train_losses']))

Epoch 3/1000
33572/33572 - 427s - loss: 0.3340 - val_loss: 0.3105
Epoch 4/1000
33572/33572 - 428s - loss: 0.3089 - val_loss: 0.2963
Epoch 5/1000
33572/33572 - 421s - loss: 0.2905 - val_loss: 0.2705
Epoch 6/1000
33572/33572 - 429s - loss: 0.2676 - val_loss: 0.2494
Epoch 7/1000
33572/33572 - 425s - loss: 0.2523 - val_loss: 0.2392
Epoch 8/1000
33572/33572 - 432s - loss: 0.2430 - val_loss: 0.2319
Epoch 9/1000
33572/33572 - 420s - loss: 0.2363 - val_loss: 0.2261
Epoch 10/1000
33572/33572 - 437s - loss: 0.2311 - val_loss: 0.2230
Epoch 11/1000
33572/33572 - 431s - loss: 0.2270 - val_loss: 0.2179
Epoch 12/1000
33572/33572 - 436s - loss: 0.2236 - val_loss: 0.2169
Epoch 13/1000
33572/33572 - 435s - loss: 0.2209 - val_loss: 0.2150
Epoch 14/1000
33572/33572 - 435s - loss: 0.2188 - val_loss: 0.2123
Epoch 15/1000
33572/33572 - 430s - loss: 0.2169 - val_loss: 0.2109
Epoch 16/1000
33572/33572 - 428s - loss: 0.2155 - val_loss: 0.2100
Epoch 17/1000
33572/33572 - 423s - loss: 0.2142 - val_loss: 0.2096
Ep

In [ ]:
# L7 0.28
# L3 0.239
# L3 with logE, angle correction, and pulse flags 0.205